In [1]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using Macro
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JuMP

  Activating project at `~/Macro`


In [2]:
case_path = @__DIR__
println("###### ###### ######")
println("Running case at $(case_path)")

system = Macro.load_system(case_path)

model = Macro.generate_model(system)

Macro.set_optimizer(model, Gurobi.Optimizer);

Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

Macro.optimize!(model)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement


┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from system/nodes.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading JSON data from assets/power_lines.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from assets/storage.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from assets/thermal.js

system.settings.Scaling = false

┌ Info: Loading CSV data from assets/availability.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14


┌ Info: Starting model generation
└ @ Macro /home/al3792/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /home/al3792/Macro/src/generate_model.jl:17
┌ Info: Defining available capacity
└ @ Macro /home/al3792/Macro/src/generate_model.jl:20
┌ Info: Generating planning model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:23
┌ Info: Generating operational model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:26
┌ Info: Model generation complete, it took 18.825639009475708 seconds
└ @ Macro /home/al3792/Macro/src/generate_model.jl:31


Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 648263 rows, 394234 columns and 2311593 nonzeros
Model fingerprint: 0x350420bf
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 166996 rows and 61875 columns
Presolve time: 1.08s
Presolved: 481267 rows, 332359 columns, 1943112 n

In [3]:
case = "capacity_with_co2_cap"

capacity_results = Macro.get_optimal_asset_capacity(system)
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results)
println("wrote results")

wrote results


In [5]:
co2_nodes = Macro.get_nodes_sametype(system.locations, CO2)

total_co2_emissions_df = DataFrame(
    node_1_emissions = Macro.value(sum(co2_nodes[1].operation_expr[:emissions])),
    node_2_emissions = Macro.value(sum(co2_nodes[2].operation_expr[:emissions])),
    node_3_emissions = Macro.value(sum(co2_nodes[3].operation_expr[:emissions]))
)
Macro.write_csv(joinpath(results_dir, case * "_co2_emissions.csv"), total_co2_emissions_df)

┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP /home/al3792/.julia/packages/JuMP/i68GU/src/operators.jl:282


"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/results/capacity_with_co2_cap_co2_emissions.csv"

In [6]:
total_co2_emissions_df

Row,node_1_emissions,node_2_emissions,node_3_emissions
,Float64,Float64,Float64
1,18000.0,25000.0,25000.0
